In [11]:
#https://towardsdatascience.com/discover-insights-from-your-instagram-business-account-with-facebook-graph-api-and-python-81d20ee2e751

#https://www.powermyanalytics.com/connectors/instagram-insights/fields/

# Import Libraries
import requests
import json
import datetime
import pandas as pd
import datetime
import os
from conexao_banco import conexao_aws
from dateutil.relativedelta import relativedelta
import pytz
utc=pytz.UTC

# Define Parameters Dictionary
params = dict()
params['access_token'] = os.getenv('access_token_instagram')
# TOKEN DE ACESSO
params['client_id'] =os.getenv('client_id_instagram')
              # not an actual client id
params['client_secret'] = os.getenv('client_secret_instagram')    # not an actual client secret
params['graph_domain'] = 'https://graph.facebook.com'
params['graph_version'] = 'v12.0'
params['endpoint_base'] = params['graph_domain'] + '/' + params['graph_version'] + '/'
params['page_id'] = os.getenv('page_id_instagram')                 # not an actual page id
params['instagram_account_id'] =  os.getenv('account_id_instagram')  # not an actual instagram business account id
params['ig_username'] = 'edufinance'

In [12]:
endpointParams = dict()
endpointParams['input_token'] = params['access_token']
endpointParams['access_token'] = params['access_token']

# Define URL
url = params['graph_domain'] + '/debug_token'

# Requests Data
data = requests.get(url, endpointParams)
access_token_data = json.loads(data.content)


In [73]:
print("Token Expires: ", datetime.datetime.fromtimestamp(access_token_data['data']['expires_at']))

#2022-06-11 13:24:27

Token Expires:  2022-06-11 13:24:27


In [14]:
# Define URL
url = params['endpoint_base'] + 'oauth/access_token'

# Define Endpoint Parameters
endpointParams = dict() 
endpointParams['grant_type'] = 'fb_exchange_token'
endpointParams['client_id'] = params['client_id']
endpointParams['client_secret'] = params['client_secret']
endpointParams['fb_exchange_token'] = params['access_token']

# Requests Data
data = requests.get(url, endpointParams )
long_lived_token = json.loads(data.content)


In [69]:
# Define URL
url = params['endpoint_base'] + params['instagram_account_id'] + '/media'


# Define Endpoint Parameters
endpointParams = dict()
endpointParams['fields'] = 'id,caption,media_product_type,media_type,permalink,timestamp,username,like_count,comments_count'
endpointParams['access_token'] = params['access_token']

# Requests Data
data = requests.get(url, endpointParams )
basic_insight = json.loads(data.content)
basic_insight1 = basic_insight
all_basics_insights = pd.DataFrame(basic_insight['data'])

lista_df_insights = []
lista_df_insights.append(all_basics_insights)

fotos_ids = []
for i in range(0,len(basic_insight['data'])):
    fotos_ids.append(basic_insight['data'][i]['id']) #pegando os id's das primeiras paginas

Existe_proxima_pagina = True
while Existe_proxima_pagina:
    try:
        url_next = basic_insight['paging']['next']
        data_next = requests.get(url_next)
        basic_insight = json.loads(data_next.content)

        for i in range(0,len(basic_insight['data'])):
            fotos_ids.append(basic_insight['data'][i]['id']) #pegando os id's das nextpages

    except Exception as e:
        Existe_proxima_pagina = False
        break
    lista_df_insights.append(pd.DataFrame(basic_insight['data']))


all_basics_insights = pd.concat(lista_df_insights, ignore_index=True)
all_basics_insights.columns = ['Id', 'Legenda', 'Local_da_midia','Tipo_da_midia', 'Link', 'UTC_da_postagem', 'Username', 'Likes', 'Comentarios']

all_basics_insights['UTC_da_postagem'] = pd.to_datetime(all_basics_insights['UTC_da_postagem'], format='%Y-%m-%d')

all_basics_insights

,Id,Legenda,Local_da_midia,Tipo_da_midia,Link,UTC_da_postagem,Username,Likes,Comentarios
0,17944093174915810,Quer conhecer a operação da Weg por completo e...,FEED,CAROUSEL_ALBUM,https://www.instagram.com/p/CcWTB1Ep60i/,2022-04-14 22:34:04+00:00,edufinance,264,1
1,17877890945657449,Quem são os bilionários russos?\n\nVladimir Po...,FEED,IMAGE,https://www.instagram.com/p/CcREihCMU6O/,2022-04-12 21:51:16+00:00,edufinance,513,8
2,17968988470576750,WEG indo em busca de novos mercados?\n\nAno pa...,FEED,CAROUSEL_ALBUM,https://www.instagram.com/p/CcOfxD_piIQ/,2022-04-11 21:51:27+00:00,edufinance,690,10
3,18286358728027596,1 - A WEG ganha dinheiro com 4 áreas de negóci...,FEED,VIDEO,https://www.instagram.com/tv/CcG1axqvojV/,2022-04-08 22:27:23+00:00,edufinance,165,1
4,17944815718794790,"Em 2016, o Brasil chegou ao maior número de ag...",FEED,IMAGE,https://www.instagram.com/p/CcEM1iTLc_h/,2022-04-07 21:53:38+00:00,edufinance,589,11
...,...,...,...,...,...,...,...,...,...
704,17870533915388382,Esses são os melhores bancos do Brasil em 2019...,FEED,CAROUSEL_ALBUM,https://www.instagram.com/p/ByQ5C7alGnJ/,2019-06-03 22:12:16+00:00,edufinance,63,9
705,17855657146430995,Esses são os maiores players de Medicina Diagn...,FEED,IMAGE,https://www.instagram.com/p/ByIqzb7Bajv/,2019-05-31 17:33:54+00:00,edufinance,54,7
706,17869391695377304,O BOVA11 é um dos ETFs mais famosos da Bolsa d...,FEED,CAROUSEL_ALBUM,https://www.instagram.com/p/ByGlIuqFWDe/,2019-05-30 22:05:54+00:00,edufinance,70,3
707,17991524527242111,Essas são as 10 empresas brasileiras mais vali...,FEED,IMAGE,https://www.instagram.com/p/ByD57qLhOBq/,2019-05-29 21:09:54+00:00,edufinance,92,11


In [17]:
media_insight = []

# Loop Over 'Media ID'
cont = 0
for id in all_basics_insights['Id'].to_list():
    cont = cont + 1
    params['latest_media_id'] = id
    # Define URL
    url = params['endpoint_base'] + params['latest_media_id'] + '/insights'

    # Define Endpoint Parameters
    endpointParams = dict() 
    endpointParams['metric'] = 'engagement,impressions,reach,saved,video_views'
    endpointParams['access_token'] = params['access_token'] 
    
    # Requests Data
    data = requests.get(url, endpointParams )
    json_data_temp = json.loads(data.content)
  
    try:
        media_insight.append(list(json_data_temp['data']))

    except:
        pass

    if cont > 100:
        break

In [18]:
data = datetime.datetime.now()

# Initialize Empty Container
engagement_list = []
impressions_list = []
reach_list = []
saved_list = []
video_views = []
date_time = []
numero = []

# Loop Over Insights to Fill Container
cont = 0
for i,insight in enumerate(media_insight):
    cont = cont + 1
    numero.append(i)
    engagement_list.append(insight[0]['values'][0]['value'])
    impressions_list.append(insight[1]['values'][0]['value'])
    reach_list.append(insight[2]['values'][0]['value'])
    saved_list.append(insight[3]['values'][0]['value'])
    video_views.append(insight[4]['values'][0]['value'])
    date_time.append(data)

    if cont > 7:
        break

# Create DataFrame
df_media_insight = pd.DataFrame(list(zip(numero,date_time,fotos_ids, engagement_list, impressions_list, reach_list, saved_list,video_views)), columns =['Numero','Data_de_extracao','Id' , 'Engajamento', 'Impressoes', 'Alcance', 'Salvos','Visualizacoes_dos_videos'])

df_media_insight

#pegar quando um post foi postado e pegar a conatgem da semana desse post

,Numero,Data_de_extracao,Id,Engajamento,Impressoes,Alcance,Salvos,Visualizacoes_dos_videos
0,0,2022-04-18 09:46:42.480344,17944093174915810,338,8996,7084,74,0
1,1,2022-04-18 09:46:42.480344,17877890945657449,741,17635,14402,41,0
2,2,2022-04-18 09:46:42.480344,17968988470576750,185,4476,3913,20,1445
3,3,2022-04-18 09:46:42.480344,18286358728027596,928,16568,13284,109,0
4,4,2022-04-18 09:46:42.480344,17944815718794790,677,15585,12140,54,0
5,5,2022-04-18 09:46:42.480344,17924719304119152,367,11795,10931,37,6036
6,6,2022-04-18 09:46:42.480344,17941058050859795,606,14521,12223,43,0
7,7,2022-04-18 09:46:42.480344,18152450800244534,382,11334,8872,34,0


In [58]:
all_insights = all_basics_insights.merge(df_media_insight)

all_insights = all_insights[['Id','Data_de_extracao','UTC_da_postagem','Likes','Comentarios',"Engajamento",'Impressoes','Alcance','Salvos','Visualizacoes_dos_videos','Tipo_da_midia','Username','Link','Legenda','Local_da_midia']]

all_insights #só as que estavam no instagram depois da conversão para conta comercial

usuario_sql = os.getenv('usuario_sql')
senha_sql = os.getenv('senha_sql')
    
aws = conexao_aws(senha = senha_sql, usuario=usuario_sql, nome_do_banco='redes_sociais')
aws.iniciar_conexao()


# all_insights.to_sql('informacoes_midias_instagram', aws.engine, index=False, if_exists='append', chunksize=10000, method='multi')

In [57]:
#IDENTIFICA SE TEM MIDIA E SE ESSA MIDIA FOI POSTADA HÁ MENOS DE 15 MINUTOS E ADICIONA AO BANCO DE DADOS



#>>>> BANCO DE DADOS RODANDO A CADA 1 MINUTO <<<<<<<<<<


ids_midias = all_insights['Id']#chamar os ids na hora de rodar o programa
utc_postagem_midias = all_insights['UTC_da_postagem'] #pegando a data de postagem dos ids
ids_midias_na_base = (pd.read_sql(f'''SELECT * FROM informacoes_midias_instagram ''', con= aws.engine)).tail(10) #pegando os ids do bancos de dados)
utc_midias_na_base = pd.read_sql(f'''SELECT * FROM informacoes_midias_instagram ''', con= aws.engine) #pegando os UTC_da_postagem do bancos de dados


df_estatico = pd.DataFrame()
for id_foto in ids_midias: #ids pegos pelo request na api
    if int(id_foto) not in ids_midias_na_base['Id'].to_list():#se id não estiver na base
        adicionar = all_insights[all_insights.Id == id_foto]
        df_estatico = pd.concat([df_estatico, adicionar], axis=0, ignore_index=True)
        print('Id',id_foto,' não estava na base, foi adicionado')



tempo_agora = datetime.datetime.utcnow()  #tranformando na mesma formatação do pandas
df_estatico = pd.DataFrame()
for i, id_foto in enumerate(ids_midias_na_base['Id'].to_list()): #COMPARAR OS IDS COM OS IDS DA BASE
    if tempo_agora < (ids_midias_na_base.iloc[i, 3] + relativedelta(weeks=1)): #o tempo de agora for maior que o tempo da publicação + 15 minutos então
        adicionar = ids_midias_na_base[ids_midias_na_base.Id == id_foto] 
        df_estatico = df_estatico.append(adicionar)
        print(i,id_foto)



df_estatico

#df_estatico.to_sql('informacoes_midias_instagram', aws.engine, index=False, if_exists='append', chunksize=10000, method='multi')



2 17944093174915810
3 17877890945657449
4 17968988470576750


,Numero,Id,Data_de_extracao,UTC_da_postagem,Likes,Comentarios,Engajamento,Impressoes,Alcance,Salvos,Visualizacoes_dos_videos,Tipo_da_midia,Username,Link,Legenda,Local_da_midia
743,752,17944093174915810,2022-04-18 09:46:42,2022-04-14 22:34:04,263,1,338,8996,7084,74,0,CAROUSEL_ALBUM,edufinance,https://www.instagram.com/p/CcWTB1Ep60i/,Quer conhecer a operação da Weg por completo e...,FEED
744,753,17877890945657449,2022-04-18 09:46:42,2022-04-12 21:51:16,513,8,741,17635,14402,41,0,IMAGE,edufinance,https://www.instagram.com/p/CcREihCMU6O/,Quem são os bilionários russos?\n\nVladimir Po...,FEED
745,754,17968988470576750,2022-04-18 09:46:42,2022-04-11 21:51:27,690,10,185,4476,3913,20,1445,CAROUSEL_ALBUM,edufinance,https://www.instagram.com/p/CcOfxD_piIQ/,WEG indo em busca de novos mercados?\n\nAno pa...,FEED


In [21]:
#https://developers.facebook.com/docs/instagram-api/reference/ig-user/insights

url = params['endpoint_base'] + params['instagram_account_id'] + '/insights'

# Define Endpoint Parameters
endpointParams = dict()
endpointParams['metric'] = 'impressions, follower_count, reach, email_contacts, phone_call_clicks, text_message_clicks, get_directions_clicks, website_clicks, profile_views'  
#follower_count só conta um periodo de 30 dias

endpointParams['period'] = 'day'
endpointParams['access_token'] = params['access_token']

# Requests Data
data = requests.get(url, endpointParams )
json_account_metrics = json.loads(data.content)

metrics_name = []
metrics_value = []
metrics_time = []
date_time = []

data = datetime.datetime.now()


metrics_name.append(json_account_metrics['data'][0]['title'])
metrics_value.append(json_account_metrics['data'][0]['values'][0]['value'])
metrics_time.append(json_account_metrics['data'][0]['values'][0]['end_time'])
date_time.append(data)
metrics_name.append(json_account_metrics['data'][0]['title'])
metrics_value.append(json_account_metrics['data'][0]['values'][1]['value'])
metrics_time.append(json_account_metrics['data'][0]['values'][1]['end_time'])
date_time.append(data)

cont = 0
Existe_pagina_anterior = True
while Existe_pagina_anterior:
    cont = cont + 1
    try:
        if cont > 14: #CADA PAGINA TEM 2 DIAS E SÓ PODE 30 DE FOLLOWER_COUNT
            endpointParams['metric'] = 'impressions, reach, email_contacts, phone_call_clicks, text_message_clicks, get_directions_clicks, website_clicks, profile_views'
        url_previous = json_account_metrics['paging']['previous']

        data_previous = requests.get(url_previous,endpointParams)
        json_account_metrics = json.loads(data_previous.content)
        for metrics in json_account_metrics['data']:

            try:
                metrics_name.append(metrics['title'])
                metrics_value.append(metrics['values'][0]['value'])
                metrics_time.append(metrics['values'][0]['end_time'])
                date_time.append(data)
                metrics_name.append(metrics['title'])
                metrics_value.append(metrics['values'][1]['value'])
                metrics_time.append(metrics['values'][1]['end_time'])
                date_time.append(data)
                
            except:
                pass

    except Exception as e:
        Existe_pagina = False
        break
    
    if cont == 1: #fazendo um contador, pois só precisamos do ultimo dia atualizado(já pegamos a base de dados)
        break
    
# Create DataFrame
# date_time = pd.DataFrame(date_time)

df_account_metrics = pd.DataFrame(list(zip(metrics_time,metrics_name,metrics_value)),columns =['UTC_do_dia','Nome' , 'Valor'])

df_account_metrics['UTC_do_dia'] = pd.to_datetime(df_account_metrics['UTC_do_dia'], format='%Y-%m-%d')

df_account_metrics = df_account_metrics.pivot(index="UTC_do_dia",columns="Nome",values="Valor")

df_account_metrics['Data_de_extracao'] = [datetime.datetime.now()] * len(df_account_metrics)


df_account_metrics = df_account_metrics[['Data_de_extracao','Alcance','Impressões','Visualizações do perfil', 'Número de seguidores', 'Cliques no site']]

df_account_metrics = df_account_metrics.rename(columns={'Impressões': 'Impressoes'})
df_account_metrics = df_account_metrics.rename(columns={'Visualizações do perfil': 'Visualizacoes_do_perfil'})
df_account_metrics = df_account_metrics.rename(columns={'Número de seguidores': 'Numero_de_seguidores'})
df_account_metrics = df_account_metrics.rename(columns={'Cliques no site': 'Cliques_no_site'})


df_account_metrics.dropna(inplace=True) #excluindo colunas com valores vazios

df_account_metrics = df_account_metrics.tail(1)

df_account_metrics

usuario_sql = os.getenv('usuario_sql')
senha_sql = os.getenv('senha_sql')

aws = conexao_aws(senha = senha_sql, usuario=usuario_sql, nome_do_banco='redes_sociais')
aws.iniciar_conexao()


df_account_metrics.to_sql('informacoes_conta_instagram', aws.engine, index=True, if_exists='append', chunksize=10000, method='multi')



In [ ]:
####################### PARA DEPOIS( LOCALIZAÇÃO, GENERO, IDADE) DO PERFIL COMPLETO ############################################

url = params['endpoint_base'] + params['instagram_account_id'] + '/insights'

# Define Endpoint Parameters
endpointParams = dict()
endpointParams['metric'] = 'audience_gender_age, audience_locale, audience_country, audience_city, online_followers'

# #não funciobam com period=day

endpointParams['period'] = 'lifetime'
endpointParams['access_token'] = params['access_token']

# Requests Data
data = requests.get(url, endpointParams )
json_account_metrics_lifetime = json.loads(data.content)

#json_account_metrics_lifetime

#necessário??????

In [22]:
url = params['endpoint_base'] + params['instagram_account_id'] 

# Define Endpoint Parameters
endpointParams = dict()
endpointParams['fields'] = 'id,ig_id,name,username,followers_count,follows_count,media_count'
endpointParams['access_token'] = params['access_token']

data = requests.get(url, endpointParams )
json_account_fields= json.loads(data.content)

date_time = []
generic_followers = []
generic_follows = []
generic_midia_count = []
generic_account_id = []


data = datetime.datetime.now()

date_time.append(data)
generic_account_id.append(json_account_fields['id'])
generic_followers.append(json_account_fields['followers_count'])
generic_follows.append(json_account_fields['follows_count'])
generic_midia_count.append(json_account_fields['media_count'])

df_metrics_accounts_generics = pd.DataFrame(list(zip(date_time,generic_followers, generic_follows, generic_midia_count,)), columns =['Data_de_extracao','Seguidores','Seguindo','Midias'])


df_metrics_accounts_generics = df_metrics_accounts_generics[['Data_de_extracao','Seguidores','Seguindo','Midias']]

df_metrics_accounts_generics

usuario_sql = os.getenv('usuario_sql')
senha_sql = os.getenv('senha_sql')
    
aws = conexao_aws(senha = senha_sql, usuario=usuario_sql, nome_do_banco='redes_sociais')
aws.iniciar_conexao()


df_metrics_accounts_generics.to_sql('informacoes_iniciais_conta_instagram', aws.engine, index=False, if_exists='append', chunksize=10000, method='multi')

In [70]:
url = params['endpoint_base'] + params['instagram_account_id'] + '/stories'

# Define Endpoint Parameters
endpointParams = dict()
endpointParams['access_token'] = params['access_token']

# Requests Data
data = requests.get(url, endpointParams )
json_stories_id = json.loads(data.content)
json_stories_id = pd.DataFrame(json_stories_id['data'])

names_insights_stories = []
values_insights_stories = []
ids_insights_stories = []
try:
    for id in json_stories_id['id'].to_list():
        url = params['endpoint_base'] + id + '/insights'

        # Define Endpoint Parameters
        endpointParams = dict()
        endpointParams['metric'] = 'impressions, reach, taps_forward, taps_back, exits, replies'
        endpointParams['access_token'] = params['access_token']

        # Requests Data
        data = requests.get(url, endpointParams)
        json_stories_insights = json.loads(data.content)
        for insights in json_stories_insights['data']:
            
            names_insights_stories.append( insights['name'])
            values_insights_stories.append( insights['values'][0]['value'])
            valor_id_tratado = insights['id']
            ids_insights_stories.append(valor_id_tratado[0:17])

        

        df_insights_stories = pd.DataFrame(list(zip(ids_insights_stories,names_insights_stories,values_insights_stories)), columns =['Id','Nome','Valor'])


    df_insights_stories = df_insights_stories.pivot(index='Id',columns='Nome',values='Valor') 

    df_insights_stories['Data_de_extracao'] = [datetime.datetime.now()] * len(df_insights_stories)

    df_insights_stories.columns = ['Saidas', 'Impressoes', 'Alcance', 'Respostas', 'Toques_para_voltar', 'Toques_para_avancar', 'Data_de_extracao']

    df_insights_stories = df_insights_stories[['Data_de_extracao','Impressoes','Alcance','Respostas','Saidas','Toques_para_voltar','Toques_para_avancar']]
    
except:
    print("Nenhum storie encontrado")


df_insights_stories


,Data_de_extracao,Impressoes,Alcance,Respostas,Saidas,Toques_para_voltar,Toques_para_avancar
Id,,,,,,,
17952686827818245,2022-04-18 13:02:09.329806,143,142,0,45,1,85


In [71]:
id_storie_fields = []
like_count_storie_fields = []
media_product_type_fields = []
media_type_fields = []
media_url_fields = []
permalink_fields = []
shortcode_fields =[]
timestamp_fields = []
username_fields = []
caption_fields = []

try:

    for id in json_stories_id['id'].to_list():
        url = params['endpoint_base'] + id   

        # Define Endpoint Parameters
        endpointParams = dict()
        endpointParams['fields'] = 'caption,id,like_count,media_product_type,media_type,media_url,owner,permalink,shortcode,thumbnail_url,timestamp,username'
        endpointParams['access_token'] = params['access_token']

        # Requests Data
        data = requests.get(url, endpointParams)
        json_stories_fields = json.loads(data.content)
    
        id_storie_fields.append(id)
        like_count_storie_fields.append(json_stories_fields['like_count'])
        media_product_type_fields.append(json_stories_fields['media_product_type'])
        media_type_fields.append(json_stories_fields['media_type'])
        media_url_fields.append(json_stories_fields['media_url'])
        permalink_fields.append(json_stories_fields['permalink'])
        shortcode_fields.append(json_stories_fields['shortcode'])
        timestamp_fields.append(json_stories_fields['timestamp'])
        username_fields.append(json_stories_fields['username'])

        try:
            caption_fields.append(json_stories_fields['caption'])
        except:
            caption_fields.append('Nan')
except:
    print("Nenhum storie encontrado")

df_fields_stories = pd.DataFrame(list(zip(id_storie_fields, like_count_storie_fields, media_product_type_fields, media_type_fields, media_url_fields, permalink_fields,timestamp_fields, username_fields, caption_fields)), columns =['Id','Likes','Local_da_midia','Tipo_da_midia', 'Thumbnail_url','Link','UTC_da_postagem','Username', 'Legenda'])

df_fields_stories

,Id,Likes,Local_da_midia,Tipo_da_midia,Thumbnail_url,Link,UTC_da_postagem,Username,Legenda
0,17952686827818245,0,STORY,VIDEO,https://video.cdninstagram.com/o1/v/t16/f1/m51...,https://instagram.com/stories/edufinance/28192...,2022-04-18T15:23:08+0000,edufinance,Nan


In [72]:
try:    
    all_stories_informations = pd.merge(df_fields_stories,  df_insights_stories, how='left', on = 'Id')

    all_stories_informations = all_stories_informations[['Id','UTC_da_postagem','Data_de_extracao','Likes','Impressoes','Alcance','Respostas','Saidas','Toques_para_voltar','Toques_para_avancar','Tipo_da_midia','Username','Link', 'Thumbnail_url','Local_da_midia','Legenda']]


    all_stories_informations['Id'] = all_stories_informations['Id'].astype(int)

    all_stories_informations['UTC_da_postagem'] = pd.to_datetime(all_stories_informations['UTC_da_postagem'], format='%Y-%m-%d')

    # all_stories_informations['Id']

    aws = conexao_aws(senha = senha_sql, usuario=usuario_sql, nome_do_banco='redes_sociais')
    aws.iniciar_conexao()


    # all_stories_informations.to_sql('informacoes_stories_instagram', aws.engine, index=False, if_exists='append', chunksize=10000, method='multi')
except:
    print("Nenhum storie encontrado")